In [ ]:
import xarray
import numpy as np

import hvplot.xarray

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import geoviews as gv

import matplotlib.pyplot as plt
%matplotlib inline
import cmocean.cm as cmo

In [ ]:
import os
home_top_dir=os.environ["HOME"]
print("Assuming your files are stored somewhere similar to mine,\nwith $HOME/COAWST and $HOME/seahorce-scidac/ROMSX\nwhere your environment variable $HOME reads as "+home_top_dir+"\n")
print(home_top_dir)

Load ROMS data from upwelling test
==================================

In [ ]:
ds_roms = xarray.open_dataset(home_top_dir+"/COAWST/roms_his.nc", chunks={"ocean_time": 21})
ds_roms

In [ ]:

if ds_roms.Vtransform == 1:
    Zo_rho = ds_roms.hc * (ds_roms.s_rho - ds_roms.Cs_r) + ds_roms.Cs_r * ds_roms.h
    z_rho = Zo_rho + ds_roms.zeta * (1 + Zo_rho / ds_roms.h)
    Zo_w = ds_roms.hc * (ds_roms.s_w - ds_roms.Cs_w) + ds_roms.Cs_w * ds_roms.h
    z_w = Zo_w + ds_roms.zeta * (1 + Zo_w / ds_roms.h)
    # also include z coordinates with mean sea level (constant over time)
    z_rho0 = Zo_rho
    z_w0 = Zo_w
elif ds_roms.Vtransform == 2:
    Zo_rho = (ds_roms.hc * ds_roms.s_rho + ds_roms.Cs_r * ds_roms.h) / (ds_roms.hc + ds_roms.h)
    z_rho = ds_roms.zeta + (ds_roms.zeta + ds_roms.h) * Zo_rho
    Zo_w = (ds_roms.hc * ds_roms.s_w + ds_roms.Cs_w * ds_roms.h) / (ds_roms.hc + ds_roms.h)
    z_w = ds_roms.zeta + (ds_roms.zeta + ds_roms.h) * Zo_w
    # also include z coordinates with mean sea level (constant over time)
    z_rho0 = ds_roms.h * Zo_rho
    z_w0 = ds_roms.h * Zo_w

ds_roms.coords["z_w"] = z_w.transpose('ocean_time', 's_w', 'eta_rho', 'xi_rho')
ds_roms.coords["z_rho"] = z_rho.transpose('ocean_time', 's_rho', 'eta_rho', 'xi_rho')
ds_roms.coords["z_rho0"] = z_rho0.transpose('s_rho', 'eta_rho', 'xi_rho')
ds_roms.coords["z_w0"] = z_w0.transpose('s_w', 'eta_rho', 'xi_rho')



In [ ]:
section = ds_roms.temp.isel(xi_rho=slice(1,43), eta_rho=40, ocean_time=0)
plt=section.plot(x="x_rho", y="z_rho", figsize=(15, 6), clim=(25, 35))
plt.set_clim([14, 22]);

In [ ]:
print(ds_roms.temp.max().as_numpy())
print(ds_roms.temp.min().as_numpy())
print(ds_roms.x_rho)
print(ds_roms.y_rho)
print(ds_roms.z_rho)

Load ROMSX dataset from Exec/Upwelling
======================================

In [ ]:
ds = xarray.open_dataset(home_top_dir+"/seahorce-scidac/ROMSX/Exec/Upwelling/plt00000_d01.nc", chunks={"current_time": 0})
ds

In [ ]:
section = ds.temp.isel(y_rho=40)
plt=section.plot(x="x_rho", y="z_rho", figsize=(15, 6), clim=(0, 35))
plt.set_clim([14, 22]);

In [ ]:
print(ds.temp.max().as_numpy())
print(ds.temp.min().as_numpy())
print(ds.x_rho)
print(ds.y_rho)
print(ds.z_rho)